In [ ]:
# import required libs

import numpy as np
import pandas as pd
import sys
sys.path.insert(1, 'config.py')
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep

In [ ]:
# Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="c66d5a293cea48a2b218269ce954e9b5", client_secret="99c7012918c74c9dbafa4accd8f21d5b"))


In [ ]:
# read the csvs into dataframes

hot_100 = pd.read_csv("hot_100.csv")
not_hot = pd.read_csv("not_hot_100.csv")


display(hot_100.head())
display(not_hot.head())

In [ ]:
# check that the duplicates have been deleted
print(not_hot.shape)

In [ ]:
#drop cols apart from Song and Artist

not_hot = not_hot[['Song', 'Artist']]
display(not_hot.head())

In [ ]:
# check if the seach on Spotify works

song = sp.search(q="tracks: Distortion Sleep artist: Soilwork", limit=1) 
song['tracks']['items'][0]['id']

In [ ]:
# function to search for songs

def search_song(title, artist, limit=5):
    query = "tracks: "+title+" artist: "+artist
    try:
        result = sp.search(query, limit=limit)
        song_id = result['tracks']['items'][0]['id']
        return song_id
    except:
        return "not found"



In [ ]:
# use the function to return a song ID
search_song("Paint The Town Red", "Doja Cat")

In [ ]:
# add a new column 'Spotify_ID' to the hot_100 DataFrame
hot_100['Spotify_ID'] = hot_100.apply(lambda row: search_song(row['Song'], row['Artist']), axis=1)

# filter out rows with "not found" as Spotify_ID
hot_100 = hot_100[hot_100['Spotify_ID'] != "not found"]

# reset the index after filtering
hot_100.reset_index(drop=True, inplace=True)
display(hot_100)

In [ ]:
# add a new column 'Spotify_ID' to the not_hot dataframe
not_hot['Spotify_ID'] = not_hot.apply(lambda row: search_song(row['Song'], row['Artist']), axis=1)

# filter out rows with "not found" as Spotify_ID
not_hot = not_hot[not_hot['Spotify_ID'] != "not found"]

# reset the index after filtering
not_hot.reset_index(drop=True, inplace=True)
display(not_hot)

In [ ]:
display(not_hot)

In [ ]:
def get_audio_features(list_of_song_ids):
  
    """Collects audio features for a list of song IDs.

    Input:
    list_of_song_ids: A list of song IDs.

    Returns:
    pandas DataFrame containing the audio features for the IDs.
    """
    
    # split the list of song IDs into chunks of 50 to avoid hitting the rate limit.
    
    chunks = [list_of_song_ids[i:i + 50] for i in range(0, len(list_of_song_ids), 50)]

    # get audio features
    
    all_features = []
    for chunk in chunks:
        features_chunk = sp.audio_features(chunk)
        if features_chunk is not None:  # check if the result is not None.
            valid_features = [f for f in features_chunk if f is not None]
            all_features.extend(valid_features)
        else:
            print("Failed to fetch features for chunk:", chunk)

    # convert the list of dictionaries to df
    
    df_audio_features = pd.DataFrame(all_features)
    return df_audio_features

In [ ]:
# get audio features for the hot songs

df_hot_audio_features = get_audio_features(hot_100['Spotify_ID'].tolist())

# merge frames

hot_100 = pd.merge(hot_100, df_hot_audio_features, left_on='Spotify_ID', right_on='id', how='left')

#drop duplicates

hot_100.drop_duplicates(subset=['Spotify_ID'], inplace=True)

# reindex
hot_100.reset_index(drop=True, inplace=True)

display(hot_100)


In [ ]:
# get audio features for the not hot songs

df_not_hot_audio_features = get_audio_features(not_hot['Spotify_ID'].tolist())

# merge frames

not_hot = pd.merge(not_hot, df_not_hot_audio_features, left_on='Spotify_ID', right_on='id', how='left')

# drop duplicates
not_hot.drop_duplicates(subset=['Spotify_ID'], inplace=True)

# reindex
not_hot.reset_index(drop=True, inplace=True)

display(not_hot)

In [ ]:
# copy and save the dataframes with features as CSV

hot_100_featured = hot_100.copy()

hot_100_featured.to_csv('hot100.csv', index=False)

not_hot_featured = not_hot.copy()

not_hot_featured.to_csv('hot100.csv', index=False)